In [5]:
import os
import shutil

# 이미지와 라벨 경로
image_root = "/home/hyun/dev_ws/ROS_project/data/image"
label_root = "/home/hyun/dev_ws/ROS_project/data/label"

# 평탄화된 디렉토리 경로
flat_image_dir = "/home/hyun/dev_ws/ROS_project/data/flat_image"
flat_label_dir = "/home/hyun/dev_ws/ROS_project/data/flat_label"

os.makedirs(flat_image_dir, exist_ok=True)
os.makedirs(flat_label_dir, exist_ok=True)

# 이미지 평탄화
for root, _, files in os.walk(image_root):
    for file in files:
        if file.endswith(('.jpg', '.png')):
            src = os.path.join(root, file)
            dst = os.path.join(flat_image_dir, file)
            shutil.copy(src, dst)

# 라벨 평탄화
for root, _, files in os.walk(label_root):
    for file in files:
        if file.endswith('.txt'):
            src = os.path.join(root, file)
            dst = os.path.join(flat_label_dir, file)
            shutil.copy(src, dst)

print("이미지와 라벨이 평탄화되었습니다.")


이미지와 라벨이 평탄화되었습니다.


In [13]:
from glob import glob
img_list = glob('/home/hyun/dev_ws/ROS_project/data/mydata/train/images/*.jpg')
label_list = glob('/home/hyun/dev_ws/ROS_project/data/mydata/train/labels/*.txt')
print(len(img_list), len(label_list))

15392 15392


In [14]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size = 0.1,
random_state = 13)
print(len(train_img_list), len(val_img_list))

13852 1540


In [16]:
with open('/home/hyun/dev_ws/ROS_project/data/train.txt','w') as f:
    f.write('\n'.join(train_img_list)+'\n')
with open('/home/hyun/dev_ws/ROS_project/data/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list)+'\n')

In [10]:
from ultralytics import YOLO

# YOLOv8 모델 로드
model = YOLO('yolov8s.pt')

# 검증 없이 학습 실행
model.train(
    data='/home/hyun/dev_ws/ROS_project/data/data.yaml',
    epochs=2,
    imgsz=320,
    batch=16,
    name='custom_yolov8_with_freeze',
    freeze=[0, 1, 2, 3, 4, 5, 6]  # COCO 클래스에 해당하는 초기 레이어 고정
)


Ultralytics 8.3.65 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/hyun/dev_ws/ROS_project/data/data.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_with_freeze, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2, 3, 4, 5, 6], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /home/hyun/dev_ws/ROS_project/data/mydata/train/labels... 13852 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13852/13852 [00:04<00:00, 3218.20it/s]


train: New cache created: /home/hyun/dev_ws/ROS_project/data/mydata/train/labels.cache


val: Scanning /home/hyun/dev_ws/ROS_project/data/mydata/train/labels... 1540 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1540/1540 [00:00<00:00, 1884.27it/s]

val: New cache created: /home/hyun/dev_ws/ROS_project/data/mydata/train/labels.cache


...

                   all       1540       1540       0.99      0.997      0.995       0.83



2 epochs completed in 0.043 hours.
Optimizer stripped from runs/detect/custom_yolov8_with_freeze/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/custom_yolov8_with_freeze/weights/best.pt, 22.5MB

Validating runs/detect/custom_yolov8_with_freeze/weights/best.pt...
Ultralytics 8.3.65 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
Model summary (fused): 168 layers, 11,157,318 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:06<00:00,  7.24it/s]


                   all       1540       1540       0.99      0.997      0.995      0.831
                  fire        525        525      0.981      0.994      0.994      0.735
                 smoke       1015       1015          1          1      0.995      0.927
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/custom_yolov8_with_freeze


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b01e6b4c4d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [11]:
from ultralytics import YOLO

# 학습된 모델 로드
model = YOLO('/home/hyun/dev_ws/ROS_project/data/src/runs/detect/custom_yolov8_with_freeze/weights/best.pt')  # 학습된 가중치 경로

# 추론할 이미지 경로
image_path = '/home/hyun/dev_ws/test.webp'

# 이미지 추론
results = model(image_path)

# 결과를 저장할 디렉토리 설정
output_dir = '/home/hyun/dev_ws/ROS_project/data/output'

# 디렉토리가 없으면 생성
import os
os.makedirs(output_dir, exist_ok=True)

# 각 결과를 저장
for idx, result in enumerate(results):
    # 저장할 파일명 설정
    save_path = os.path.join(output_dir, f"result_{idx}.jpg")
    result.plot(save=True, filename=save_path)  # 결과를 저장

# 저장된 결과 확인
print(f"Results saved to {output_dir}")



image 1/1 /home/hyun/dev_ws/test.webp: 320x320 2 smokes, 6.0ms
Speed: 1.4ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
Results saved to /home/hyun/dev_ws/ROS_project/data/output


In [1]:
from ultralytics import YOLO
import cv2
import os

# 학습된 YOLO 모델 로드
model = YOLO('/home/hyun/dev_ws/ROS_project/data/src/runs/detect/custom_yolov8_with_freeze/weights/best.pt')  # 학습된 가중치 경로

# OpenCV로 카메라 열기
cap = cv2.VideoCapture(0)  # 0은 기본 카메라, 다른 카메라를 사용하려면 번호 변경

# 결과 저장 디렉토리 설정
output_dir = '/home/hyun/dev_ws/ROS_project/data/output'
os.makedirs(output_dir, exist_ok=True)

# 실시간 영상 처리
while cap.isOpened():
    ret, frame = cap.read()  # 카메라에서 프레임 읽기
    if not ret:
        print("카메라에서 프레임을 읽을 수 없습니다.")
        break

    # YOLO 모델로 추론
    results = model(frame)

    # 결과를 이미지에 그리기
    annotated_frame = results[0].plot()

    # 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 결과를 파일로 저장 (필요한 경우)
    save_path = os.path.join(output_dir, "realtime_result.jpg")
    cv2.imwrite(save_path, annotated_frame)

# 자원 정리
cap.release()
cv2.destroyAllWindows()



0: 256x320 (no detections), 25.2ms
Speed: 3.0ms preprocess, 25.2ms inference, 20.9ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.2ms
Speed: 0.8ms preprocess, 5.2ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.8ms
Speed: 1.5ms preprocess, 5.8ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.3ms
Speed: 0.7ms preprocess, 5.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 6.2ms
Speed: 1.9ms preprocess, 6.2ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.9ms
Speed: 0.7ms preprocess, 5.9ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 6.1ms
Speed: 1.0ms preprocess, 6.1ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 320)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/hyun/venv/torch_venv/lib/python3.12/site-packages/cv2/qt/plugins"



0: 256x320 (no detections), 5.9ms
Speed: 0.9ms preprocess, 5.9ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.5ms
Speed: 0.9ms preprocess, 5.5ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 6.7ms
Speed: 0.9ms preprocess, 6.7ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.3ms
Speed: 0.8ms preprocess, 5.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.4ms
Speed: 0.7ms preprocess, 5.4ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.2ms
Speed: 0.6ms preprocess, 5.2ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.3ms
Speed: 0.8ms preprocess, 5.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 5.3ms
Speed: 0.7ms preprocess, 5.3ms inference, 0.5ms 